In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import scipy
from numpy import asarray
import numpy as np
from numpy import unravel_index
from math import exp
import random



In [ ]:
train_female = '/kaggle/input/gender-classification-dataset/dataset/train/female'
train_male = '/kaggle/input/gender-classification-dataset/dataset/train/male'

In [ ]:
imagesf = os.listdir(train_female)
imagesm = os.listdir(train_male)

Pooling layer reduces the size of filter layer allowing the model to train faster.
**Prevents overfitting** by dropping unwanted values in filter layer.

In [ ]:

def max_pool(y):
    (t,a,b) = y.shape
    output = np.zeros((int(a/2),int(b/2)))
    list_max = []
    list_max_index = []
    
    output_a = np.zeros((t,int(a/2),int(b/2)))
    list_max_a = []
    list_max_index_a = []
    i = 0
    j = 0
    k = 1
    while k <= t:
        while i< (int(a/2)+1):
            j = 0
            while j < (int(b/2)+1):
                a1 = y[i:i+2,j:j+2].max()
                list_max.append(a1)
                list_max_index.append(unravel_index(w.argmax(), w.shape))
                
                j += 2
            i += 2
        list_max_a.append(list_max)
        list_max_index_a.append(list_max_index)
        k += 1
    output_a = np.reshape(np.asarray(list_max_a),(t,int(a/2),int(b/2)))
    return output_a

The filter which matches the most with the given input region will produce an output which will be higher in magnitude.


In [ ]:

def convolve(m1,m2):
    
    (a,b,c) = m1.shape #2,3,3.
    (w,x,y,z) = m2.shape  #2,2,2
    output = np.zeros((w,b-y+1,c-z+1))
    for g in range(w):
        for k in range(a): 
            for i in range(b-y+1):
                for j in range(c-z+1):
                    output[g][i][j] += np.sum(np.multiply(m1[k,i:i+y,j:j+z],m2[k,:,:]))
                
    return output

In [ ]:
def full_convolve(m1,m2):
    (a,b,c) = m1.shape
    (x,y,z) = m2.shape
    padded_matrix = np.zeros((a,b+2*y-2,c+2*z-2))
    padded_matrix[y-1:y-1+b,z-1:z-1+c] = m1
    convolve(padded_matrix,m2)

In [ ]:
def same_convolve(m1,m2):
    #kernel has to be a 3*3 matrix
    
    (a,b,c) = m1.shape
    (x,y,z) = m2.shape
    padded_matrix = np.zeros((a,b+2*y-2,c+2*z-2))
    padded_matrix[y-1:y-1+b,z-1:z-1+c] = m1
    convolve(padded_matrix,m2)

In [ ]:
def cross_correlation(mat1,mat2):
    mat2 = flip_kernel(mat2)
    out = convolve(mat1,mat2)
    return out
    
    
    
def flip_kernel(mat2):
    
    size = mat.shape
    kernel_f = []
    for i in range(size[0]):
        for j in range(size[1]):
            kernel_f[i,size[1]-1-j] = mat[size[0]-1-i,j]
    return kernel_f

In [ ]:
def forward_propagation(input_mat,kernel_size):
    a,b,c,d = kernel_size
    kernel = np.random.rand(a,b,c,d)

    x = convolve(input_mat,kernel)
    print(x.shape)
    bias = np.random.rand(*x.shape)
    
    y = x + bias
    return y
    
def backward_propagation(input_mat,kernel,bias,learning_rate,output_gradient):
    grad_kernel = convolve(input_mat,output_gradient)
    grad_bias =  output_gradient
    grad_input =  full_convolve(output_gradient,kernel)
    
    kernel -= learning_rate*grad_kernel
    bias -= learning_rate*grad_bias

In [ ]:

def forward_max(self,input_mat):
    maxpool(input_mat)
    
def backward_max(self,matrix):
    (a,b) = matrix.shape
    reverse = np.zeros((a*2,b*2))
    for i in list_max:
        for j in list_max_index:
            reverse[j]=i

In [ ]:
def sigmoid_F(x):
    z = np.zeros(x.shape)
    for i in range(x.shape[0]):
        z[i] = ( 1/(1 + np.exp(-x[i])))
    
    return z
def sigmoid_B(x):
    return (sigmoid_F(x) * (1-sigmoid_F(x)))
#sigmoid activation function

In [ ]:
#flattening
def flattening(mat):
    for k in range(len(mat)):
        b = mat[k].size
        mat[k].reshape(b,1)
    return mat
def unflatten(mat):
    mat.reshape() #fix

In [ ]:
def image_to_array(imagesnames,path,m_or_f,train_or_test):
    
    images_matrix=[]
    
    
    for item in imagesnames:
        image = cv2.imread(path+'/'+item)
        
        #image = image.resize((160, 160, 3))
        
        image_array = asarray(image)
        #print(image_array)
        
        image_array = np.concatenate((image_array[:,:,0],image_array[:,:,1],image_array[:,:,2])).reshape(3, *np.array(image_array[:,:,0]).shape)
        
        
        if train_or_test =="test":
            images_matrix.append(image_array)
        
        if m_or_f == 1 and train_or_test =="train":
            images_matrix.append([image_array,1])
        
        elif m_or_f == 0 and train_or_test =="train":
            images_matrix.append([image_array,0])
    return images_matrix

In [ ]:

train_female_ = image_to_array(imagesf,train_female,1,"train")
train_male_ = image_to_array(imagesm,train_male,0,"train")

In [ ]:
#CNN
        
def Net_F(input_mat):
    i = 1
    while i <= 5:
        for j in range(len(input_mat)):
            
            kernel_size = 64
            a =forward_propagation(input_mat[j],(kernel_size,input_mat[j].shape[0],3,3))
            b= sigmoid_F(a)
            c = forward_propagation(b,(kernel_size,b.shape[0],3,3))
            d = sigmoid_F(c)
            input_mat[j] = forward_max(d)
        i += 1
        kernel_size *= 2
        Net_F(input_mat)     
    return input_mat
    
def Net_B(input_mat,output_error_grad):
    kernel_error_grad = np.zeros(kernel.shape)
    bias_error_grad = np.zeros(input_mat.shape)
    input_error_grad = np.zeros(input_mat.shape)
        
    kernel_error_grad = cross_correlation(input_mat,output_error_gradient)
    bias_error_grad = output_error_gradient
        
    
    for i in range(kernel.shape[2]):
        for j in range(kernel.shape[2]):
            kernel_error_grad[i, j] = cross-correlation(input_mat[j], output_error_grad[i])
            input_gradient[j] += full_convolve(output_error_grad[i], kernel[i, j])

    kernel -= learning_rate * kernel_error_grad
    bias -= learning_rate * bias_error_grad
    return input_error_grad

In [ ]:
def tanh(x):
    return (exp(x)-exp(-x))/(exp(x)+exp(-x))

In [ ]:
def grad_tanh(x):
    return(1-(tanh(x))^2)

In [ ]:

def neural_net_f(input_array,num_output):
    (m,n) = input_array.shape
    bias = np.random.rand(1,num_output)
    weight = np.random.rand(n,num_output)
    y1 = np.dot(input_array,weight) + bias
        
    return y1
    
def neural_net_b(output_error_grad,learning_rate):
    input_error_grad = np.dot(output_error_grad , weight.T)
    weight_error_grad = np.dot(input_array.T,output_error_grad)
    bias_error_grad = output_error_grad
        
    weight -= learning_rate * weight_error_grad
    bias -= learning_rate * output_error_grad
    return input_error_grad

In [ ]:
def dense(flattened):
    d1 = []
    d2 = []
    d3 = []
    for m in range(len(flattened)):
        #dense layer 1
        d1[m] = neural_network_f(flattened[m],8192)
        d1[m] = sigmoid_F(d1[m])
        
        #dense layer 2
        d2[m] = neural_network_f(d1[m],1024)
        d2[m] = sigmoid_F(d2[m])
        
        #dense layer 3
        d3[m] = neural_network_f(d2[m],2)
        d3[m] = sigmoid_F(d3[m])
    
    return d3

In [ ]:
def activation_f(input_mat):
        
    output_mat = tanh(input_mat)
    return output_mat
    
def activation_b(output_error, learning_rate):
    return tanh_grad(input_mat) * output_error

In [ ]:
#error
def mean_squared_error(y, y_predicted):
    return np.mean(np.power(y-y_predicted, 2))
def mean_squared_error_derv(y, y_predicted):
    return (2*(y_predicted-y)/y.size)

In [ ]:
#train
def train(train_female_,train_male_):
    train = train_female_ + train_male_
    random.shuffle(train)
    
    y = [item[1] for item in train]
    train_data =[item[0] for item in train]
    
    
    
    for i in range(2): #epochs
        cnn = Net_F(train_data)
        print(cnn)
        flat = flattening(cnn)
        y_pred = dense(flat)
        print(y_pred)
        error = mean_squared_error(y,y_pred)
        
        #backward
        output_e_grad = mean_squared_error_derv(y,y_pred)
        print(output_e_grad)
        k = 0
        while k<3:
            output_e_grad = sigmoid_B(output_e_grad)
            print("yes")
            output_e_grad = neural_net_b(output_e_grad)
            k += 1
        Net_B(unflatten(output_e_grad))

In [ ]:
train(train_female_,train_male_)